In [2]:
import scipy.stats as stats
import numpy as np
import scikit_posthocs as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
conf_energy_dict = {}
conf_time_dict = {}
lang_energy_dict = {}
lang_time_dict = {}
lang_num_dict = {}

all_energy_list = []
avg_energy_list = []

df_energy_dict = {}

with open("..\\data\\Data.csv", "r") as f:
    lines = f.readlines()
    for line in lines:
        if line.startswith("Group"):
            continue
        line = line.strip().split(",")
        if line[1] == "" and line[3] == "":
            continue
        if line[1] == "" and line[3] != "":
            continue
        lang = line[0]
        conf = line[1]
        energy = float(line[3])
        time = float(line[5])
        if conf not in conf_energy_dict:
            conf_energy_dict[conf] = []
        
        conf_energy_dict[conf].append(energy)
        if conf not in conf_time_dict:
            conf_time_dict[conf] = []        
        conf_time_dict[conf].append(time)

        if lang not in lang_energy_dict:
            lang_energy_dict[lang] = []
        lang_energy_dict[lang].append(energy)
        
        if lang not in lang_time_dict:
            lang_time_dict[lang] = []
        lang_time_dict[lang].append(time)
        
        if lang not in lang_num_dict:
            lang_num_dict[lang] = 0
        lang_num_dict[lang] += 1

In [4]:
# Normality test

list_of_normality = []
list_of_non_normality = []
conf_normality_dict = {}


for config, data in conf_energy_dict.items():
    stat, p_value = stats.shapiro(data)
    conf_normality_dict[config] = p_value
    if p_value < 0.05:
        list_of_non_normality.append(config)
    else:
        list_of_normality.append(config)
print("Normality: ", list_of_normality)
print("Non Normality: ", list_of_non_normality)
print("Shapiro-Wilk test p-values: ", conf_normality_dict)

Normality:  ['go-default', 'go-jemalloc', 'go-pgo', 'node-default', 'node-jemalloc', 'dotnet-default', 'dotnet-jemalloc', 'dotnet-quick-jit-false', 'python-default', 'python-jemalloc', 'python-default-4-workers', 'python-jemalloc-4-workers', 'ruby-default', 'ruby-jemalloc', 'ruby-yjit', 'rust-default', 'rust-PGO', 'rust-jemalloc']
Non Normality:  ['java-default', 'java-jemalloc', 'node-jitless']
Shapiro-Wilk test p-values:  {'go-default': 0.5283713340759277, 'go-jemalloc': 0.3671122193336487, 'go-pgo': 0.6453002095222473, 'java-default': 0.0008717963937669992, 'java-jemalloc': 0.0007519629434682429, 'node-default': 0.8487940430641174, 'node-jemalloc': 0.842570424079895, 'node-jitless': 0.0011928870808333158, 'dotnet-default': 0.4383963942527771, 'dotnet-jemalloc': 0.5290932059288025, 'dotnet-quick-jit-false': 0.681451141834259, 'python-default': 0.17892540991306305, 'python-jemalloc': 0.9191370606422424, 'python-default-4-workers': 0.43419596552848816, 'python-jemalloc-4-workers': 0.06

In [5]:
p_value_dict = {}

In [6]:
# For the normally distributed data, we use student's t-test
# Specifically we use the independent two-sample t-test, as we have
# 2 independent samples each time (default and another config)
for lang in lang_num_dict.keys():
    default = []
    for config, data in conf_energy_dict.items():
        if lang in config:
            if config == str(lang) + '-' + "default":
                default = data
                continue
            if config in list_of_non_normality:
                continue
            stat, p_value = stats.ttest_ind(default, data)
            print(f"t-test between {lang} default and {config}: t-statistic = {stat}, p-value = {p_value}")
            p_value_dict[config] = p_value
            


t-test between go default and go-jemalloc: t-statistic = 0.613468279565349, p-value = 0.5566027668884561
t-test between go default and go-pgo: t-statistic = 2.543293108246538, p-value = 0.03453329263634111
t-test between node default and node-jemalloc: t-statistic = -1.5448503555857662, p-value = 0.16096294518515705
t-test between dotnet default and dotnet-jemalloc: t-statistic = -2.8397065129777763, p-value = 0.021821816779441606
t-test between dotnet default and dotnet-quick-jit-false: t-statistic = -5.4942369754952045, p-value = 0.0005776595005664528
t-test between python default and python-jemalloc: t-statistic = -12.009603959846947, p-value = 2.130835631669278e-06
t-test between python default and python-default-4-workers: t-statistic = -5.263697328572447, p-value = 0.0007610839321377533
t-test between python default and python-jemalloc-4-workers: t-statistic = -3.8340930567249045, p-value = 0.004988836627025367
t-test between ruby default and ruby-jemalloc: t-statistic = 14.50080

In [7]:
# For the non normally distributed data, we use mann-whitney U test
for lang in lang_num_dict.keys():
    default = []
    for config, data in conf_energy_dict.items():
        if lang in config:
            if "default" in config:
                default = data
                continue
            if config in list_of_normality:
                continue
            stat, p_value = stats.mannwhitneyu(default, data)
            print(f"Comparison between {lang} default and {config}: U-statistic = {stat}, p-value = {p_value}")
            p_value_dict[config] = p_value

Comparison between java default and java-jemalloc: U-statistic = 7.0, p-value = 0.30952380952380953
Comparison between node default and node-jitless: U-statistic = 0.0, p-value = 0.007936507936507936


In [8]:
print(p_value_dict)

{'go-jemalloc': 0.5566027668884561, 'go-pgo': 0.03453329263634111, 'node-jemalloc': 0.16096294518515705, 'dotnet-jemalloc': 0.021821816779441606, 'dotnet-quick-jit-false': 0.0005776595005664528, 'python-jemalloc': 2.130835631669278e-06, 'python-default-4-workers': 0.0007610839321377533, 'python-jemalloc-4-workers': 0.004988836627025367, 'ruby-jemalloc': 5.008233197323181e-07, 'ruby-yjit': 1.2108815590678066e-10, 'rust-PGO': 0.6558446042130086, 'rust-jemalloc': 0.07824113982427737, 'java-jemalloc': 0.30952380952380953, 'node-jitless': 0.007936507936507936}
